In [86]:
# Python standard libraries
import math

# Packages for computation and modelling
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Self-defined packages
import utils

# Magic command to reload packages whenever we run any later cells
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
BOARD = [["A",    "B",  "C",   "D",    "E",    "F",     "G",    "H"    ],
         ["I",    "J",  "K",   "L",    "M",    "N",     "O",    "P"    ],
         ["Q",    "R",  "S",   "T",    "U",    "V",     "W",    "X"    ],
         ["Y",    "Z",  "Sp",  "1",    "2",    "3",     "4",    "5"    ],
         ["6",    "7",  "8",   "9",    "0",    "Prd",   "Ret",  "Bs"   ],
         ["?",    ",",  ";",   "\\",   "/",    "+",     "-",    "Alt"  ],
         ["Ctrl", "=",  "Del", "Home", "UpAw", "End",   "PgUp", "Shft" ],
         ["Save", "'",  "F2",  "LfAw", "DnAw", "RtAw",  "PgDn", "Pause"],
         ["Caps", "F5", "Tab", "EC",   "Esc",  "email", "!",    "Sleep"]]
BOARD  = np.array(BOARD)
N_ROWS = BOARD.shape[0]  # number of rows
N_COLS = BOARD.shape[1]  # number of columns
M = N_ROWS * N_COLS      # the number of chars on the board

In [78]:
paradigm       = 'RC'  # display paradigm ('RC', 'CB', or 'RD')
NUM_TIMESTAMPS = 195   # number of timestamps in each window to record signals
EPOCH_SIZE     = 15    # required number of features in every epoch
CORE_CHANNELS  = ('EEG_Fz', 'EEG_Cz',  'EEG_P3',  'EEG_Pz',
                  'EEG_P4', 'EEG_PO7', 'EEG_PO8', 'EEG_Oz')
NUM_CORE_CHANNELS  = len(CORE_CHANNELS)  # number of core eletrodes
NUM_TRAIN_WORDS = 5 # number of training words for one participant
NUM_TEST_WORDS  = 5 # number of testing words for one participant

obj = 1 # the index of experiment object (participant)
obj = str(obj) if obj >= 10 else '0'+str(obj)
directory = '/Users/zionsheng/Desktop/Duke/Research/P300-BCI/BCI_data/EDFData-StudyA'
obj_directory = directory + f'/A{obj}/SE001'

In [11]:
train_features,train_response = utils.load_data(dir=obj_directory,
                                                obj=obj,
                                                num_timestamps=NUM_TIMESTAMPS,
                                                epoch_size=EPOCH_SIZE,
                                                num_channels=NUM_CORE_CHANNELS,
                                                type=paradigm,
                                                mode='train',
                                                num_words=NUM_TRAIN_WORDS)

In [12]:
test_features,test_response   = utils.load_data(dir=obj_directory,
                                                obj=obj,
                                                num_timestamps=NUM_TIMESTAMPS,
                                                epoch_size=EPOCH_SIZE,
                                                num_channels=NUM_CORE_CHANNELS,
                                                type=paradigm,
                                                mode='test',
                                                num_words=NUM_TEST_WORDS)

In [13]:
train_features.shape

(4284, 120)

In [79]:
d = 120
sim = 20
small_n    = np.arange(50, 115, 2)
around     = np.arange(115, 124, 1)
large_n    = np.arange(124, 200, 2)
larger_n   = np.arange(200, 300, 5)
largest_n  = np.arange(500, train_features.shape[0], 50)
n_vals = np.concatenate([small_n, around, large_n, larger_n, largest_n])
M = n_vals.shape[0]
print(f'We set {M} candidates for sample size (n).')

We set 176 candidates for sample size (n).


In [91]:
def run_simulation(train_features, train_response, test_features, test_response,
                   n_vals, num_sim):
    M = n_vals.shape[0]
    train_mse = np.zeros((M, num_sim))
    test_mse  = np.zeros((M, num_sim))

    for i in range(M):
        for j in range(num_sim):
            n = n_vals[i]
            indices = np.random.choice(train_features.shape[0], n, replace=False)
            sub_train_features = train_features[indices, ]
            sub_train_response = train_response[indices, ]
            clf = LogisticRegression(penalty=None)
            clf.fit(sub_train_features, sub_train_response)
            train_pred = clf.predict(train_features)
            test_pred  = clf.predict(test_features)
            train_mse[i,j] = log_loss(train_response, train_pred)
            test_mse[i,j]  = log_loss(test_response, test_pred)

    return train_mse, test_mse

In [ ]:
train_mse, test_mse = run_simulation(train_features, train_response,
                                     test_features, test_response,
                                     n_vals, sim)

In [ ]:
train_mse

In [ ]:
avgs_train_loss = np.mean(train_mse, 1)
avgs_test_loss  = np.mean(test_mse, 1)

plt.title("Test Risk vs Sample Size (D=120)")
plt.xlabel("# Training Samples")
plt.ylabel("MSE")
plt.ylim(0, 13)
plt.xlim(0, 600)
plt.plot(n_vals, avgs_test_loss,  lw=2, color='blue', label='test loss')
plt.plot(n_vals, avgs_train_loss, lw=2, color='red', label='train loss')
plt.axvline(x=d, color='black', ls='--')
plt.legend(loc="best")
plt.show()